<a href="https://colab.research.google.com/github/zooveevolbeido/portfolio/blob/main/decision_tree_classifier_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.datasets import load_breast_cancer
import math

df = load_iris()

#print(df)


X = df['data']

y = df['target']

#print(X.shape)
#print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=0)

#print(len(X_train))
#print(len(X_test))
y_train = y_train.reshape((len(y_train),1))

data = np.concatenate((X_train, y_train),axis=1)

#print(z)



def divideLR(data, feature, val):

    dataL =  data[ data[:,feature] <= val ]
    dataR =  data[ data[:,feature] >  val]

    return dataL, dataR



def chooseParam(data):

    ## remove all duplicate values in features to choose split point
    removed = []
    dataT = data[:,:-1].transpose()

    for i in range(len(dataT)):
        removed.append([*set(dataT[i])]   )

    orig_gini = gini_impurity(data)
    next_gini = 1000

    #fea = 0
    #va = removed[0][0]

    for f in range(len(removed)):
        for v in range(0, len(removed[i]) ):  #, math.ceil(len(removed[i])/7)    ):

            if gini_impurity( divideLR(data, f, v)[0] )*  (len(divideLR(data, f, v)[0])/len(data) )  +  gini_impurity( divideLR(data, f, v)[1] )*  (len(divideLR(data, f, v)[1])/len(data) ) < next_gini:
                next_gini = gini_impurity( divideLR(data, f, v)[0] )* ( len(divideLR(data, f, v)[0])/len(data) )  +  gini_impurity( divideLR(data, f, v)[1] )*  (len(divideLR(data, f, v)[1])/len(data) )
                splitFeature = f
                splitVal = v
                dataLeft = divideLR(data, f, v)[0]
                dataRight = divideLR(data, f, v)[1]

                #if next_gini<0.4:
                 #   return splitFeature, splitVal, dataLeft, dataRight, next_gini



                if orig_gini - next_gini > 0.2: #and len(dataLeft)>19 and len(dataRight)>19 : #next_gini < orig_gini:
                    return splitFeature, splitVal, dataLeft, dataRight, next_gini


    return False





def gini_impurity(data):

    dct = {}

    denomLen = len(data)

    onlyY = data[:,-1]

    for i in onlyY:
        if i not in dct:
            dct[i] = 1
        else:
            dct[i] += 1

    con = list( dct.values() )

    giniVal = 1

    for i in range(len(con)):
        giniVal -= (con[i]/denomLen)**2

    return giniVal



def find_mostfreq(data):
    maxFreq = 0
    k=0

    dct = {}

    denomLen = len(data)

    onlyY = data[:,-1]

    for i in onlyY:
        if i not in dct:
            dct[i] = 1
        else:
            dct[i] += 1

    for i in dct:
        if dct[i] > maxFreq:
            k = i
            maxFreq = dct[i]

    return k





def helper(data, dct):

    if chooseParam(data)!=False:
        #dct = {'params':[chooseParam(data)[0],chooseParam(data)[1], chooseParam(data)[2], chooseParam(data)[3],find_mostfreq(data)], 'left':None, 'right':None}
        dct = {'splitF': chooseParam(data)[0], 'splitV':chooseParam(data)[1], 'dataLeft':chooseParam(data)[2], 'dataRight':chooseParam(data)[3], 'mostLabel':find_mostfreq(data), 'left':None, 'right':None}
        #nxt_dataL = dct['params'][2]
        nxt_dataL = dct['dataLeft']
        #nxt_dataR = dct['params'][3]
        nxt_dataR = dct['dataRight']




    return dct, nxt_dataL, nxt_dataR




def makeTree(data, dct, dep):

    if dct==None:
        #dct = {'params':[chooseParam(data)[0],chooseParam(data)[1], chooseParam(data)[2], chooseParam(data)[3],find_mostfreq(data)], 'left':None, 'right':None}
        #nxt_dataL = dct['params'][2]
        #nxt_dataR = dct['params'][3]
        dct = {'splitF': chooseParam(data)[0], 'splitV':chooseParam(data)[1], 'dataLeft':chooseParam(data)[2], 'dataRight':chooseParam(data)[3], 'mostLabel':find_mostfreq(data), 'left':None, 'right':None}

    if dep <3:
        if  chooseParam(helper(data, dct)[2])  and dct['right'] == None:
            dct['right'] = helper( helper(data, dct)[2], dct )[0]
            dep+=1
            makeTree(helper(data, dct)[2], dct['right'], dep)

        elif chooseParam(helper(data, dct)[2]) == False:
            dct['right'] = {'splitF':None, 'splitV':None, 'dataLeft':None, 'dataRight':None, 'mostLabel':find_mostfreq(helper(data, dct)[2]), 'left':None, 'right':None}
            #return

        if chooseParam(helper(data, dct)[1])  and dct['left'] == None :
            dct['left'] = helper( helper(data, dct)[1], dct )[0]
            dep+=1
            makeTree(helper(data, dct)[1], dct['left'], dep)
        elif chooseParam(helper(data, dct)[1]) == False :
            dct['left'] = {'splitF':None, 'splitV':None, 'dataLeft':None, 'dataRight':None, 'mostLabel':find_mostfreq(helper(data, dct)[1]), 'left':None, 'right':None}



    return dct






diction = makeTree(data, None, 0)
#print(ans['left']['right']['params'][3])

#print(diction)



def sol(diction, X_test):
    con = []

    for i in range(len(X_test)):
        con.append(exam(diction, X_test[i]))

    return con

#print(X_test)




def exam(diction, te):


    if diction['splitF']==None:
        #con.append(diction['mostLabel'])
        #exam(diction, X_test, i+1, con)
        return diction['mostLabel']


    if diction['splitF'] != None and te[  diction['splitF'] ] <= diction['splitV']:
        return exam(diction['left'], te)

    elif diction['splitF'] != None  and te[  diction['splitF'] ] > diction['splitV']:
        return exam(diction['right'], te)


y_pred = sol(diction, X_test) #exam(diction, X_test, 0, [])

print(y_pred)

print()

print(y_test)

print()

#print(y_pred[0]==y_test[0])


count = 0
for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
        count+=1


print(count)
print()
print(len(y_test))
print()
print('accuracy', count/len(y_test))

print()

####################################################################

import time
startTime = time.time()

#####your python script#####

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))




[2.0, 1.0, 0.0, 2.0, 0.0, 2.0, 0.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]

28

30

accuracy 0.9333333333333333

Execution time in seconds: 3.0279159545898438e-05
